This is my first attempt at GANs!

I will try to generate logos for later use with the bill-forge project.

Here we go!

In [1]:
ls

datalab/


In [2]:
# Imports
import keras
from keras import layers, Input, models, optimizers
import numpy as np
import datetime
import tensorflow as tf
import os
import h5py
from keras.preprocessing import image
from google.colab import files

Using TensorFlow backend.


In [0]:
!mkdir images

In [4]:
# Download LLD - Large Logo Dataset from https://data.vision.ee.ethz.ch/sagea/lld/
!wget https://data.vision.ee.ethz.ch/sagea/lld/data/LLD-icon-sharp.hdf5 #(32 x 32)

--2018-05-31 18:59:34--  https://data.vision.ee.ethz.ch/sagea/lld/data/LLD-icon-sharp.hdf5
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 313576929 (299M)
Saving to: ‘LLD-icon-sharp.hdf5’

LLD-icon-sharp.hdf5  76%[==============>     ] 229.75M  5.10MB/s    eta 14s    

LLD-icon-sharp.hdf5 100%[===================>] 299.05M  4.86MB/s    in 59s     

2018-05-31 19:00:33 (5.06 MB/s) - ‘LLD-icon-sharp.hdf5’ saved [313576929/313576929]



In [5]:
ls

datalab/  images/  LLD-icon-sharp.hdf5


**GAN Overview**
![GAN in a nutshell](https://deeplearning4j.org/img/GANs.png)

In [6]:
# GAN Generator Network
latent_dim = 32
one_hot_dim = 32
height = 32
width = 32
channels = 3

generator_input = Input(shape=(latent_dim,), name='random_latent_vector')

generator_labels = Input(shape=(one_hot_dim,), name='random_1hot_class')

generator_combined = layers.concatenate([generator_input, generator_labels], name='gen_input')

x = layers.Dense((1024 + one_hot_dim) * 2 * 2)(generator_combined)
x = layers.LeakyReLU()(x)
x = layers.Reshape((2, 2, 1024 + one_hot_dim))(x)

x = layers.Conv2DTranspose(512 + one_hot_dim, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256 + one_hot_dim, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(128 + one_hot_dim, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(64 + one_hot_dim, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, 5, activation='tanh', padding='same', name='gen_image')(x)

generator = models.Model(input=[generator_input, generator_labels], outputs=[x])

generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
random_latent_vector (InputLaye (None, 32)           0                                            
__________________________________________________________________________________________________
random_1hot_class (InputLayer)  (None, 32)           0                                            
__________________________________________________________________________________________________
gen_input (Concatenate)         (None, 64)           0           random_latent_vector[0][0]       
                                                                 random_1hot_class[0][0]          
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4224)         274560      gen_input[0][0]                  
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`


In [7]:
# GAN Discriminator Network
discriminator_input = Input(shape=(height, width, channels), name='image_input')

discriminator_labels = Input(shape=(height, width, one_hot_dim), name='image_class')

discriminator_combined = layers.concatenate([discriminator_input, discriminator_labels], name='disc_input')

x = layers.Conv2D(64, 1)(discriminator_combined)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(512, 5, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(1024, 5, strides=2, padding='same')(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.6)(x)

x = layers.Dense(1, activation='sigmoid', name='real_fake_pred')(x)

discriminator = models.Model(inputs=[discriminator_input, discriminator_labels], outputs=[x])

discriminator.summary()

discriminator_optimizer = optimizers.RMSprop(
  lr=0.0002,
  clipvalue=1.0,
  decay=1e-8)

discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_input (InputLayer)        (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
image_class (InputLayer)        (None, 32, 32, 32)   0                                            
__________________________________________________________________________________________________
disc_input (Concatenate)        (None, 32, 32, 35)   0           image_input[0][0]                
                                                                 image_class[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   2304        disc_input[0][0]                 
__________

In [8]:
# GAN Adversarial Network
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
gan_labels = Input(shape=(one_hot_dim,))
gan_output_labels = Input(shape=(height, width, one_hot_dim))

gan_output = discriminator(inputs=[generator(inputs=[gan_input, gan_labels]), gan_output_labels])

gan = models.Model(inputs=[gan_input, gan_labels, gan_output_labels], outputs=[gan_output])

gan.summary()

gan_optimizer = optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 32, 32, 3)    20097251    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32, 32, 32)   0                                            
__________

In [0]:
# import logo data
hdf5_file = h5py.File('LLD-icon-sharp.hdf5', 'r')
labels = hdf5_file['labels/resnet/rc_32'][:]

new_images = hdf5_file['data'][:]
new_images = np.moveaxis(new_images, 1, -1)
new_images = new_images.reshape((new_images.shape[0],) + (height, width, channels)).astype('float32') / 255.
new_images = new_images[0:len(labels)]

In [179]:
print(one_hot_labels.shape)
print(new_images.shape)

(221312, 32)
(221312, 32, 32, 3)


In [0]:
# GAN Training & Generation
iterations = 10000
batch_size = 128
save_dir = 'images'

start = 0
for step in range(iterations):
  
  stop = start + batch_size
  
  # Sample random points in the latent space
  random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
  
  # Create random labels for the random points
  random_input_labels = np.random.randint(one_hot_dim, size=batch_size)
  random_one_hot = np.zeros((batch_size, one_hot_dim))
  for i, e in enumerate(random_input_labels):
    random_one_hot[i][e] = 1
  
  # Generate new images (ie. decode to fake images)
  generated_images = generator.predict([random_latent_vectors, random_one_hot])
  
  # Get the class tensor for the fake images
  generated_image_classes = np.zeros((batch_size, height, width, one_hot_dim))
  for i, e in enumerate(random_input_labels):
    generated_image_classes[i][e][:][:] = 1
   
  # Grab real images & their labels
  real_images = new_images[start: stop]
  real_labels = labels[start: stop]
  
  # Get the class tensor for the real images
  real_images_classes = np.zeros((batch_size, height, width, one_hot_dim))
  for i, e in enumerate(real_labels):
    real_images_classes[i][e][:][:] = 1
  
  # Combine fake images with real images
  combined_images = np.concatenate([generated_images, real_images])
  combined_labels = np.concatenate([generated_image_classes, real_images_classes])
  
  # Assemble labels discriminating real from fake images and add some noise to the discriminator labels
  disc_labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
  disc_labels += 0.05 * np.random.random(disc_labels.shape)
  
  # Train the discriminator
  d_loss = discriminator.train_on_batch([combined_images, combined_labels], disc_labels)
  
  # Sample random points in the latent space
  random_latent_vectors_gan = np.random.normal(size=(batch_size, latent_dim)) 
  
  # Create random labels for the random points
  random_input_labels_gan = np.random.randint(one_hot_dim, size=batch_size)
  random_one_hot_gan = np.zeros((batch_size, one_hot_dim))
  for i, e in enumerate(random_input_labels_gan):
    random_one_hot_gan[i][e] = 1
  
  generated_image_classes_gan = np.zeros((batch_size, height, width, one_hot_dim))
  for i, e in enumerate(random_input_labels_gan):
    generated_image_classes_gan[i][e][:][:] = 1
  
  # Assemble labels that say "all real images"
  misleading_targets = np.zeros((batch_size, 1))
  
  # Train the generator (via the gan model, where the discriminator weights are frozen)
  a_loss = gan.train_on_batch([random_latent_vectors_gan, random_one_hot_gan, generated_image_classes_gan], misleading_targets)

  start += batch_size
  
  if start > len(new_images) - batch_size:
    start = 0
    
  # write to output losses & to file images
  if step % 100 == 0:
    
    gan.save_weights('gan.h5')
    
    print('step:', step)
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)
    
    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, 'generated_logo' + str(step) + '.png'))
    
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir, 'real_logo' + str(step) + '.png'))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


step: 0
discriminator loss: 0.6882568
adversarial loss: 0.4862601
step: 100
discriminator loss: 0.34599057
adversarial loss: 5.816887


In [0]:
ls

datalab/  LLD-icon/         LLD-icon-sharp.hdf5    logs/
gan.h5    LLD-icon.hdf5     LLD-icon-sharp.hdf5.1  ngrok*
images/   LLD-icon_PKL.zip  LLD-icon-sharp.hdf5.2  ngrok-stable-linux-amd64.zip


In [0]:
!tar -zcvf images.tar.gz images/
files.download('images.tar.gz')

In [0]:
# Do not run unless I want to reset the machine!
!kill -9 -1